In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import lightgbm as lgb
import pandas as pd

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "lgbm_reg.joblib"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *
from package.utils import *

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= train_start_date) & (train["date"] < validation_start_date)
is_valid = train["date"] >= validation_start_date

In [ ]:
is_train |= is_valid

In [ ]:
%%time
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/149754
dtrain = create_dataset(
    train,
    is_train,
    features,
    transformed_target,
    categorical_feature=categorical_features,
)

In [ ]:
del train
del is_train
del is_valid

In [ ]:
gc.collect()

In [ ]:
%%time
model = lgb.train(lgb_params, dtrain, valid_sets=[dtrain], verbose_eval=10)

In [ ]:
ax = lgb.plot_importance(model, importance_type="split", figsize=(16, 9))

In [ ]:
ax = lgb.plot_importance(model, importance_type="gain", figsize=(16, 9))

In [ ]:
joblib.dump(model, model_path)